In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Define the Auto-encoder
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

# Custom loss function including both reconstruction and clustering terms
class CustomLoss(nn.Module):
    def __init__(self, lambda_param, cluster_centers):
        super(CustomLoss, self).__init__()
        self.lambda_param = lambda_param
        self.cluster_centers = cluster_centers

    def forward(self, decoded, original, encoded):
        reconstruction_loss = nn.MSELoss()(decoded, original)
        clustering_loss = torch.mean(torch.min(torch.sum((encoded.unsqueeze(1) - self.cluster_centers)**2, dim=2), dim=1)[0])
        total_loss = reconstruction_loss + self.lambda_param * clustering_loss
        return total_loss

# Load and preprocess dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
data_loader = DataLoader(dataset, batch_size=256, shuffle=True)

# Training the Auto-encoder
autoencoder = AutoEncoder(32*32*3, 256, 64).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

# Pretrain autoencoder
for epoch in range(50):
    for img, _ in data_loader:
        img = img.view(img.size(0), -1).cuda()
        optimizer.zero_grad()
        decoded, encoded = autoencoder(img)
        loss = criterion(decoded, img)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Extract features for clustering
features = []
for img, _ in data_loader:
    img = img.view(img.size(0), -1).cuda()
    _, encoded = autoencoder(img)
    features.append(encoded.detach().cpu().numpy())

features = np.vstack(features)
scaler = StandardScaler()
features = scaler.fit_transform(features)
pca = PCA(n_components=50)
features_pca = pca.fit_transform(features)

# Perform K-Means clustering
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters)
clusters = kmeans.fit_predict(features_pca)
cluster_centers = torch.tensor(kmeans.cluster_centers_, dtype=torch.float).cuda()

# Fine-tune autoencoder with clustering loss
lambda_param = 0.1
custom_loss = CustomLoss(lambda_param, cluster_centers)

for epoch in range(50):
    for img, _ in data_loader:
        img = img.view(img.size(0), -1).cuda()
        optimizer.zero_grad()
        decoded, encoded = autoencoder(img)
        loss = custom_loss(decoded, img, encoded)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Extract features again
features = []
for img, _ in data_loader:
    img = img.view(img.size(0), -1).cuda()
    _, encoded = autoencoder(img)
    features.append(encoded.detach().cpu().numpy())

features = np.vstack(features)
distances = np.linalg.norm(features - cluster_centers[clusters], axis=1)

# Identify top anomalies
anomaly_indices = np.argsort(distances)[-100:]
anomalous_images = [dataset[i][0].numpy().transpose(1, 2, 0) for i in anomaly_indices]

# Visualize anomalies
fig, axes = plt.subplots(10, 10, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    ax.imshow((anomalous_images[i] * 255).astype(np.uint8))
    ax.axis('off')
plt.show()


In [ ]:
# multi-layer feature extractor
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Define a feature extractor that combines features from multiple layers
class MultiLayerFeatureExtractor(nn.Module):
    def __init__(self, pretrained_model):
        super(MultiLayerFeatureExtractor, self).__init__()
        self.layer1 = nn.Sequential(*list(pretrained_model.children())[:4])
        self.layer2 = nn.Sequential(*list(pretrained_model.children())[4:5])
        self.layer3 = nn.Sequential(*list(pretrained_model.children())[5:6])
        self.layer4 = nn.Sequential(*list(pretrained_model.children())[6:7])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        features = []
        x = self.layer1(x)
        features.append(self.avgpool(x).view(x.size(0), -1))
        x = self.layer2(x)
        features.append(self.avgpool(x).view(x.size(0), -1))
        x = self.layer3(x)
        features.append(self.avgpool(x).view(x.size(0), -1))
        x = self.layer4(x)
        features.append(self.avgpool(x).view(x.size(0), -1))
        return torch.cat(features, dim=1)

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(cifar10_train, batch_size=256, shuffle=True)

# Initialize the pretrained model and feature extractor
pretrained_model = models.resnet18(pretrained=True)
feature_extractor = MultiLayerFeatureExtractor(pretrained_model).cuda()

# Extract and combine features
features = []
for img, _ in train_loader:
    img = img.cuda()
    with torch.no_grad():
        combined_features = feature_extractor(img)
    features.append(combined_features.cpu().numpy())

features = np.vstack(features)

# Perform anomaly detection using combined features (e.g., K-Means clustering, DBSCAN, etc.)
# For simplicity, let's apply PCA and visualize the combined features
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
features_pca = pca.fit_transform(features)

plt.scatter(features_pca[:, 0], features_pca[:, 1], alpha=0.5)
plt.title("PCA of Combined Features")
plt.show()
